In [ ]:
DIR = "norms"

import os
import emcee

import numpy as np

from glob import glob
from scipy import stats
from pyazr import azure2
from multiprocess import Pool, current_process

# Restrict processes to one thread only
os.environ['OMP_NUM_THREADS'] = '1'

# Define the data labels (in AZURE2 order)
labels = ["Meyer et al. (1976) - 84.3 deg",
          "Meyer et al. (1976) - 114.5 deg",
          "Meyer et al. (1976) - 144.1 deg", 
          "LUNA HPGe (2023)", 
          "LUNA BGO (2023)",
          "Felsenkeller (2023)",
          "ATOMKI (2023)",
          "Notre Dame (2023) - 0 deg",
          "Notre Dame (2023) - 55 deg",
          "Burtebaev et al. (2008)",
          "Lamb et al. (1957)",
          "Bailey et al. (1950)",
          "Vogl et al. (1963)",
          "Rolfs et al. (1974) - 0 deg",
          "Rolfs et al. (1974) - 90 deg"]

# Define the parameters prior distributions
priors = [
    stats.norm(1.63,0.12),

    stats.uniform(2.30, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20),
    stats.uniform(-1e8,2e8),

    stats.uniform(3.45, 0.10), stats.uniform(0, 1e6), stats.uniform(-10, 20), stats.uniform(-10, 20),
    stats.uniform(-1e8,2e8), stats.uniform(-1e8,2e8),

    stats.uniform(3.50, 0.10), stats.uniform(0, 1e6),
    
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.05),
    stats.lognorm(0.069),
    stats.lognorm(0.079),
    stats.lognorm(0.10),
    stats.lognorm(0.06),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.lognorm(0.10),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100),
    stats.uniform(0, 100)
]

# Minimization variables
nsteps   = 10000        # How many steps should each walker take?
nprocs   = 7            # How many Python processes do you want to allocate?
ndim     = len(priors)  # How many parameters are you fitting?
nwalkers = 2 * ndim     # How many walkers do you want to use?

In [ ]:
# We read the .azr file and set the external capture file to speed up the calculation
azr = azure2('12c_pg.azr', nprocs=nprocs)

In [3]:
# We get the initial values from AZURE2
theta0 = azr.params
ntheta = len(theta0)

# We'll read the data from the output file since it's already in the center-of-mass frame
y = azr.cross
yerr = azr.cross_err

In [4]:
# Prior log probability
def lnPi( theta ):
    return np.sum([pi.logpdf(t) for (pi, t) in zip(priors, theta)])

# Log likelihood
def lnL( theta, proc=0 ):
    res = 0
    mu = azr.calculate( theta[:ntheta], proc=proc )
    for i in range( len( mu ) ):
        idx = ntheta + i
        res += np.sum( -0.5 * np.log(2 * np.pi * pow(yerr[i], 2) ) - 0.5 * pow((mu[i] - y[i] * theta[idx]) / (yerr[i] * theta[idx]), 2) )
    return res

# Posterior log probability
def lnP( theta ):
    try: proc = int(current_process().name.split('-')[1]) - 1 # We want to get the numbe r of the process to call the right AZURE2 port
    except: proc = 0 
    lnpi = lnPi( theta )
    if not np.isfinite( lnpi ): return -np.inf
    return lnL( theta, proc=proc ) + lnpi

In [5]:
# Get some good guesses for the initial walker positions
data = np.loadtxt( "results/{}/samples.txt".format(DIR) )
mean = np.mean( data, axis=0 )

# Prepare initial walker positions
p0 = np.zeros( (nwalkers, ndim) )
for i in range(nwalkers):
    for j in range(ndim):
        if( j not in [1, 5, 11] ): p0[i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.5, abs(mean[j]) * 1.0 ).rvs()
        else: p0[i,j] = np.sign(mean[j]) * stats.uniform( abs(mean[j]) * 0.99, abs(mean[j]) * 0.02 ).rvs()

In [ ]:
# Prepare the file to write the chains
backend = emcee.backends.HDFBackend('results/emcee/samples_{}.h5'.format(DIR)) 
backend.reset(nwalkers, len(priors))

# Run the sampling
with Pool(processes=nprocs) as pool:
    sampler = emcee.EnsembleSampler( nwalkers, ndim, lnP, pool=pool, backend=backend ) 
    state = sampler.run_mcmc( p0, nsteps, progress=True, tune=True )